# Load Data

In [1]:
import os
import pandas as pd 
import numpy as np
import logging
import csv
import re
import logging
import optparse

import dedupe
from unidecode import unidecode
from itertools import product

## Setup

In [2]:
folder = r'/filepath_to_folder/'

input_file = 'yellow_yelp_all_pypostal.csv'
output_file = 'yellow_yelp_all_pypostal_output2.csv'
settings_file = 'yellow_yelp_all_pypostal_learned_settings2'
training_file = 'yellow_yelp_all_pypostal_training2.json'

In [3]:
fp = os.path.join(folder, input_file)

In [4]:
matches_file = os.path.join(folder, 'yellow_yelp_label.csv')

In [5]:
log_level = logging.INFO
log_level = logging.DEBUG
logging.getLogger().setLevel(log_level)


## Dataframe view

In [6]:
input_df = pd.read_csv(fp, sep=',', quotechar='"', dtype={'postalcode':'str'} )

In [7]:
input_df.columns

Index(['id', 'source', 'name', 'category', 'phone', 'city', 'postalcode',
       'address', 'street', 'house_number', 'house'],
      dtype='object')

In [8]:
def get_clean_postalcode(x):
    
    if x is not None:
        subparts = str(x).split('.')
        return subparts[0]
    else:
        return None

In [9]:
input_df['postalcode'] = input_df['postalcode'].apply(lambda x: get_clean_postalcode(x))

In [ ]:
input_df.head(5)

In [10]:
categories = list(input_df['category'].unique())
categories = [x for x in categories if str(x) != 'nan']

In [11]:
#category_corpus = input_df[['name', 'category']].drop_duplicates().to_dict(orient='records')
category_corpus = input_df.drop_duplicates().to_dict(orient='records')

In [12]:
category_corpus[0]

{'id': 1,
 'source': 'yellow_pages',
 'name': 'Tao Tao Restaurant',
 'category': nan,
 'phone': '(408) 736-3731',
 'city': 'Sunnyvale ',
 'postalcode': '94086',
 'address': '175 S Murphy Ave',
 'street': 's murphy ave',
 'house_number': '175',
 'house': nan}

# Dedupe


## Import modules

In [15]:
def pre_process(val):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    try:
        val = re.sub('  +', ' ', val)
        val = re.sub('\n', ' ', val)
        val = val.strip().strip('"').strip("'").lower().strip()
        # If data is missing, indicate that by setting the value to `None`
        if not val:
            val = None
    except Exception as e:
        print(e)
    return val

In [16]:
def get_clean_data_dict(file_path):
    data_d = {}
    with open(fp) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, pre_process(v)) for (k, v) in row.items()]
            row_id = int(row['id'])
            data_d[row_id] = dict(clean_row)

    return data_d

### Get Data in needed format

In [20]:
data_dict = get_clean_data_dict(fp)

In [ ]:
data_dict

### Define the Fields for dedupe


In [ ]:
def sameOrNotComparator(field_1, field_2) :
    if field_1 and field_2 :
        if field_1 == field_2 :
            return 0
        else:
            return 1

In [17]:
fields = [
    {'field' : 'name', 'type': 'Exact'},
    {'field' : 'category', 
     'type': 'FuzzyCategorical',
     'categories': categories,
     'corpus': category_corpus,
     'has missing' : True},
    {'field' : 'name', 'type': 'String'},
    {'field' : 'address', 'type': 'Address'},
    {'field' : 'city', 'type': 'ShortString'},
    {'field': 'name', 'variable name': 'name', 'type': 'String' },
    {'field': 'postalcode', 'variable name': 'postalcode', 'type': 'Exact'},
    {'field' : 'phone', 'type': 'String'},
    {'field' : 'street', 'type': 'String', 'has missing' : True},
    {'field' : 'house_number', 'type': 'Exists', 'has missing' : True},
#    {'field' : 'house', 'type': 'String', 'has missing' : True},
    
]

### Instantiate Dedupe

In [18]:
deduper = dedupe.Dedupe(fields)

In [21]:
deduper.prepare_training(data_dict)

INFO:dedupe.canopy_index:Removing stop word ar
INFO:dedupe.canopy_index:Removing stop word ri
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word in
INFO:dedupe.canopy_index:Removing stop word n 
INFO:dedupe.canopy_index:Removing stop word ra
INFO:dedupe.canopy_index:Removing stop word  c
INFO:dedupe.canopy_index:Removing stop word nt
INFO:dedupe.canopy_index:Removing stop word  r
INFO:dedupe.canopy_index:Removing stop word es
INFO:dedupe.canopy_index:Removing stop word re
INFO:dedupe.canopy_index:Removing stop word ta
INFO:dedupe.canopy_index:Removing stop word  b
INFO:dedupe.canopy_index:Removing stop word e 
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word  s
INFO:dedupe.canopy_index:Removing stop word s 
INFO:dedupe.canopy_index:Removing stop word ca
INFO:dedupe.canopy_index:Removing stop word al
INFO:dedupe.canopy_index:Removing stop word  6
INFO:dedupe.canopy_index:Removing stop word  3
INFO:dedupe.c

In [24]:
dedupe.consoleLabel(deduper)

name : olive garden italian restaurant
category : None
address : None
city : None
postalcode : nan
phone : None
street : None
house_number : None
house : None

name : olive garden italian restaurant
category : None
address : None
city : None
postalcode : nan
phone : None
street : None
house_number : None
house : None

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


name : olive garden italian restaurant
category : None
address : None
city : None
postalcode : nan
phone : None
street : None
house_number : None
house : None

name : olive garden italian restaurant
category : None
address : None
city : None
postalcode : nan
phone : (218) 212-3793
street : None
house_number : None
house : None

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


u


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (twoGramFingerprint, name), SimplePredicate: (wholeFieldPredicate, postalcode))
name : peter piper pizza
category : None
address : None
city : None
postalcode : nan
phone : None
street : None
house_number : None
house : None

name : peter piper pizza
category : None
address : 120 s new rd
city : waco
postalcode : 76710
phone : (254) 751-1212
street : s new rd
house_number : 120
house : None

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

name : arby's
category : None
address : 9418 n interstate 20
city : sweetwater
postalcode : 79556
phone : (325) 235-1450
street : n interstate 20
house_number : 9418
house : None

name : love's travel stop
category : None
address : 9418 n interstate 20
city : sweetwater
postalcode : 79556
phone : (325) 235-1240
street : n interstate 20
house_number : 9418
house : None

17/10 positive, 9/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


name : 54th street bar and grill
category : None
address : 5715-5727 rim pass dr
city : san antonio
postalcode : 78257
phone : None
street : rim pass dr
house_number : 5715-5727
house : None

name : 54th street restaurant & drafthouse
category : None
address : 17122 w interstate 10
city : san antonio
postalcode : 78257
phone : (210) 690-5424
street : w interstate 10
house_number : 17122
house : None

17/10 positive, 10/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [ ]:
deduper2 = deduper

In [25]:
deduper.train(recall=1)

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.010000, score 0.3782722921030144
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (firstTokenPredicate, city), SimplePredicate: (twoGramFingerprint, phone))
INFO:dedupe.training:(ExistsPredicate: (Exists, house), SimplePredicate: (wholeFieldPredicate, name))
INFO:dedupe.training:(PartialPredicate: (commonTwoTokens, address, StreetName), SimplePredicate: (sameSevenCharStartPredicate, name))
INFO:dedupe.training:(SimplePredicate: (commonThreeTokens, name), SimplePredicate: (commonThreeTokens, street))


In [26]:
deduper.predicates

((SimplePredicate: (firstTokenPredicate, city),
  SimplePredicate: (twoGramFingerprint, phone)),
 (ExistsPredicate: (Exists, house),
  SimplePredicate: (wholeFieldPredicate, name)),
 (PartialPredicate: (commonTwoTokens, address, StreetName),
  SimplePredicate: (sameSevenCharStartPredicate, name)),
 (SimplePredicate: (commonThreeTokens, name),
  SimplePredicate: (commonThreeTokens, street)))

In [ ]:
with open(training_file, 'w') as tf:
    deduper.writeTraining(tf)

In [ ]:
with open(settings_file, 'wb') as sf:
    deduper.writeSettings(sf)

In [27]:
threshold = deduper.threshold(data_dict, recall_weight=1)


INFO:dedupe.blocking:10000, 1.4191872 seconds
INFO:dedupe.api:Maximum expected recall and precision
INFO:dedupe.api:recall: 0.997
INFO:dedupe.api:precision: 0.917
INFO:dedupe.api:With threshold: 0.480


In [29]:
clustered_dupes = deduper.match(data_dict, threshold)

INFO:dedupe.blocking:10000, 1.9783572 seconds


In [30]:
print('# duplicate sets', len(clustered_dupes))

# duplicate sets 2001


In [31]:
cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [data_dict[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores):
        cluster_membership[record_id] = {
            "cluster id" : cluster_id,
            "canonical representation" : canonical_rep,
            "confidence": score
        }

In [32]:
singleton_id = cluster_id + 1
with open(output_file, 'w') as f_output, open(fp) as f_input:
    writer = csv.writer(f_output)
    reader = csv.reader(f_input)

    heading_row = next(reader)
    heading_row.insert(0, 'confidence_score')
    heading_row.insert(0, 'Cluster ID')
    canonical_keys = canonical_rep.keys()
    for key in canonical_keys:
        heading_row.append('canonical_' + key)

    writer.writerow(heading_row)

    for row in reader:
        row_id = int(row[0])
        if row_id in cluster_membership:
            cluster_id = cluster_membership[row_id]["cluster id"]
            canonical_rep = cluster_membership[row_id]["canonical representation"]
            row.insert(0, cluster_membership[row_id]['confidence'])
            row.insert(0, cluster_id)
            for key in canonical_keys:
                row.append(canonical_rep[key].encode('utf8'))
        else:
            row.insert(0, None)
            row.insert(0, singleton_id)
            singleton_id += 1
            for key in canonical_keys:
                row.append(None)
        writer.writerow(row)

# Predictions

In [33]:
df = pd.read_csv(output_file)

In [ ]:
df.columns

In [34]:
df = pd.read_csv(output_file)
df.sort_values(['Cluster ID'], inplace=True)
relevant_data = df[['Cluster ID', 'confidence_score', 'source', 'id']]

predictions = []
cluster_ids = relevant_data['Cluster ID'].value_counts()
for cluster_id in cluster_ids[cluster_ids>1].index:
    
    fodors_ids = relevant_data[
        (relevant_data['Cluster ID'] == cluster_id) & 
        (relevant_data['source'] == 'yellow_pages')
    ].id.values
    zagats_ids = relevant_data[
        (relevant_data['Cluster ID'] == cluster_id) & 
        (relevant_data['source'] == 'yelp')
    ].id.values
    
    match_interim = list(product(fodors_ids, zagats_ids))
    predictions.append(match_interim)
    
m = []
for cluster in predictions: 
    for combo in cluster: 
        m.append([combo[0], combo[1]])
        
predictions = pd.DataFrame(m, columns=['yellow_pages_id', 'yelp_id'])

predictions['yp-y'] = predictions.apply(lambda row: f"{row['yellow_pages_id']}-{row['yelp_id']}", axis=1)



In [36]:
results = pd.read_csv(matches_file)
results['yp-y'] = results.apply(lambda row: f"{row['yellow_pages_id']}-{row['yelp_id']}", axis=1)


In [ ]:
results.columns

In [ ]:
results['duplicate'][0]

In [40]:
results = results[results['duplicate'] == 1]#[['yellow_pages_id', 'yelp_id', 'yp-y']]

In [42]:
len(results)

126

In [43]:
res_set = set(results['yp-y'].values.tolist())
pred_set = set(predictions['yp-y'].values.tolist())

In [44]:
tp = len(res_set & pred_set)
fn = len(res_set-pred_set)
fpos = len(pred_set-res_set)

print(f'tp: {tp} fp: {fpos} fn: {fn}')


tp: 20 fp: 2045 fn: 106
